# 데이터 불러오기

In [2]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv("./KRI-DAC_Jeju_data5.txt", sep=",", index_col=0)
print(df.shape)
df.head(2)

(273183, 13)


,Field1,YM,SIDO,SIGUNGU,FranClass,Type,Time,TotalSpent,DisSpent,NumofSpent,NumofDisSpent,POINT_X,POINT_Y
OBJECTID,,,,,,,,,,,,,
1,1,202005,제주특별자치도,제주시,영세,일반한식,00시,363000,66500,10,2,877005.9834,1.479766e+06
2,2,202005,제주특별자치도,제주시,영세,단란주점,00시,1180000,0,3,0,877005.7447,1.479816e+06


# 좌표계 변환

현재 데이터셋에는 위도와 경도에 대한 데이터가 포함되어 있는 것 같습니다. 한번 확인해봅시다.

In [4]:
print(df.POINT_X[:3])
print(df.POINT_Y[:3])

OBJECTID
1    877005.9834
2    877005.7447
3    877056.6756
Name: POINT_X, dtype: float64
OBJECTID
1    1.479766e+06
2    1.479816e+06
3    1.479616e+06
Name: POINT_Y, dtype: float64


우리가 흔히 알고 있는 좌표계와는 좀 많이 다른 것 같네요. 왜 그런 걸까요?

일반적으로 우리에게 익숙한 좌표계는 **WGS84** 좌표계입니다. 군사적인 목적으로 만들어져 현재는 전세계적으로 사용되는 일종의 표준 좌표계라고 할 수 있는데요, 지구의 질량 중심을 원점으로 해 가로축을 위도, 세로축을 경도로 지정해 지구를 격자모양으로 나눠 놓은 것이라고 할 수 있습니다. 대표적으로 구글맵이 이 좌표계를 사용합니다.

하지만 여기에서 한가지 문제가 발생하는데요, 지구는 둥글기 때문에 격자모양의 좌표계를 사용할 시 일부 지역에서 **좌표 왜곡**이 발생할 수 밖에 없다는 점입니다. 그렇기 때문에 표준화된 좌표계로는 WGS84가 사용됨에도 불구하고, 일반적으로 좌표 왜곡이 발생하는 지역들은 표준 좌표계에서 약간의 보정치를 추가해 변환한 좌표계를 사용하게 됩니다.

이러한 맥락에서 우리나라는 **UTM-K** 좌표계를 사용하고 있습니다. 현재 데이터의 지리정보도 UTM-K 좌표들인 것 같은데, 후에 구글맵 등을 이용해 지리정보를 시각화 하기 위해서는 WGS84 좌표계로 변환해줄 필요가 있습니다. 

pyproj 모듈을 이용해 이를 WGS84 좌표계로 변환하도록 하겠습니다.

좌표계 관련 참고: 

- https://rightstone032.tistory.com/7
- https://jw910911.tistory.com/51

UTM-K 좌표계 변환 공식 참고: 

- https://m.blog.naver.com/PostView.nhn?blogId=hss2864&logNo=221645763282&proxyReferer=https:%2F%2Fwww.google.com%2F


In [5]:
## 좌표계 변환 
from pyproj import Proj, transform

def get_WGS84(coordinates):
    """
    UTM-K 좌표계를 WGS로 변환해주는 함수.
    coordinates: UTM-K 좌표 리스트
    """
    proj_WGS84 = Proj(init="epsg:4326") # WGS1984
    proj_UTMK = Proj('+proj=tmerc +lat_0=38 +lon_0=127.5 +k=0.9996 +x_0=1000000 +y_0=2000000 +ellps=GRS80 + units=m +no_defs')  # UTM-K 좌표계 변환 공식
    
    long = [x[0] for x in coordinates]  # UTM-K 경도
    lat = [x[1] for x in coordinates]   # UTM-K 위도
    
    coordinates_w84 = transform(proj_UTMK, proj_WGS84, long, lat)
    
    return coordinates_w84

In [7]:
coords_utmk = list(zip(list(df.POINT_X), list(df.POINT_Y)))

# 좌표 변환
coords_wgs84 = get_WGS84(coords_utmk)

# 데이터 프레임 좌표값 수정
df["POINT_X"] = coords_wgs84[0]
df["POINT_Y"] = coords_wgs84[1]

In [8]:
df.head(3)

,Field1,YM,SIDO,SIGUNGU,FranClass,Type,Time,TotalSpent,DisSpent,NumofSpent,NumofDisSpent,POINT_X,POINT_Y
OBJECTID,,,,,,,,,,,,,
1,1,202005,제주특별자치도,제주시,영세,일반한식,00시,363000,66500,10,2,126.178894,33.302315
2,2,202005,제주특별자치도,제주시,영세,단란주점,00시,1180000,0,3,0,126.178884,33.302766
3,3,202005,제주특별자치도,제주시,중소1,편의점,00시,157670,6850,20,2,126.179458,33.300971


## 공간 시각화

In [13]:
from bokeh.io import output_notebook, show
from bokeh.models import ColumnDataSource, GMapOptions
from bokeh.layouts import layout, row
from bokeh.plotting import gmap
from bokeh.models.widgets import RadioGroup

### 업종 별 분포 시각화

In [10]:
df.head()

,Field1,YM,SIDO,SIGUNGU,FranClass,Type,Time,TotalSpent,DisSpent,NumofSpent,NumofDisSpent,POINT_X,POINT_Y
OBJECTID,,,,,,,,,,,,,
1,1,202005,제주특별자치도,제주시,영세,일반한식,00시,363000,66500,10,2,126.178894,33.302315
2,2,202005,제주특별자치도,제주시,영세,단란주점,00시,1180000,0,3,0,126.178884,33.302766
3,3,202005,제주특별자치도,제주시,중소1,편의점,00시,157670,6850,20,2,126.179458,33.300971
4,4,202005,제주특별자치도,제주시,영세,편의점,00시,46600,0,2,0,126.179430,33.302323
5,5,202005,제주특별자치도,제주시,영세,주점,00시,66000,0,2,0,126.179412,33.303224


In [17]:
X_center = df.POINT_X.mean()   # 제주도 경도 중심
Y_center = df.POINT_Y.mean()   # 제주도 위도 중심
map_options = GMapOptions(lat=Y_center, lng=X_center, map_type="roadmap", zoom=11)

plot = gmap(APIkey, map_options, title="제주도", width=900, height=600, output_backend="webgl")

source = ColumnDataSource(data = {
    "long": df[df.FranClass == "영세"].POINT_X,
    "lat": df[df.FranClass == "영세"].POINT_Y
})

plot.circle(x="long", y="lat", size=3, fill_color="blue", fill_alpha=0.5, source=source)

show(plot)
output_notebook()

Loading BokehJS ...

In [18]:
def bkapp(doc):
    
    global APIkey

    X_center = df.POINT_X.mean()   # 제주도 경도 중심
    Y_center = df.POINT_Y.mean()   # 제주도 위도 중심
    map_options = GMapOptions(lat=Y_center, lng=X_center, map_type="roadmap", zoom=11)
    
    
    # Create google map object
    plot = gmap(APIkey, map_options, title="제주도", width=900, height=600, output_backend="webgl")

    
    # column data source
    source = ColumnDataSource(data = {
        "long": df[df.FranClass == "영세"].POINT_X,
        "lat": df[df.FranClass == "영세"].POINT_Y
    })
    
    
    # Scatter Plot
    plot.circle(x="long", y="lat", size=3, fill_color="blue", fill_alpha=0.5, source=source)

    
    # Define the callback function
    categories = list(df.FranClass.unique())
    def radio_update(attr, old, new):
        active = radio.active
        source.data = {
            "long": df[df.FranClass == categories[active]].POINT_X,
            "lat": df[df.FranClass == categories[active]].POINT_Y
        }

    # Create radio button
    radio = RadioGroup(labels=categories, active=0)
    radio.on_change("active", radio_update)


    # Application on Jupyter notebook
    doc.add_root(layout([plot, radio], sizing_mode='stretch_width'))

In [19]:
output_notebook()
show(bkapp)

Loading BokehJS ...